<a href="https://colab.research.google.com/github/thiagodsd/science-avec-des/blob/master/anotacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import datetime, warnings, pickle, gc, os, math, pprint, hashlib

import numpy  as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn           as sns

from IPython.display import display

from google.colab import drive, files

In [2]:
# json = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# !pip install Boruta

In [3]:
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [8]:
# !kaggle datasets download lgmoneda/cartola-fc-brasil-scouts
# !unzip '*.zip'

  0% 0.00/322k [00:00<?, ?B/s]
100% 322k/322k [00:00<00:00, 46.6MB/s]
Archive:  cartola-fc-brasil-scouts.zip
  inflating: escalacoes.csv          
  inflating: jogadores.csv           
  inflating: jogos.csv               


# tabelas

In [11]:
for d in ['escalacoes', 'jogadores', 'jogos']:
  with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
      _df = pd.read_csv(rf'{d}.csv')
      print(d)
      display(_df)

escalacoes


,colocação,jogador,escalações,rodada,ano
0,1,Gabriel (FLA),3327349,1,2020
1,2,Pedro Geromel (GRE),3127228,1,2020
...,...,...,...,...,...
1118,19,Guarín (VAS),557526,38,2019
1119,20,Jorge Jesus (FLA),524641,38,2019


jogadores


,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,DS,G,A,SG,FS,FF,FD,FT,DD,DP,GC,CV,CA,PP,GS,FC,I,PI,rodada,ano,RB,PE
0,Elton (ATA),Sport,14.21,1,15.60,15.6,10.21,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,4.0,1,2020,NaN,NaN
1,Saravia (LAT),Internacional,13.19,1,12.20,12.2,7.19,8.0,NaN,NaN,1.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,8.0,1,2020,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12294,Wesley (MEI),Avaí,1.69,10,-0.51,2.9,0.41,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,11.0,1.0,NaN,38,2019,7.0,15.0
12295,Phelipe (GOL),Grêmio,0.78,2,-0.65,-3.3,-0.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,38,2019,NaN,1.0


jogos


,mandante,visitante,placar_mandante,placar_visitante,data,local,rodada,ano
0,Fortaleza,Athlético-PR,0.0,2.0,"SÁB, 08/08 19:00",Castelão (CE),1,2020
1,Coritiba,Internacional,0.0,1.0,"SÁB, 08/08 19:30",Couto Pereira,1,2020
...,...,...,...,...,...,...,...,...
558,Goiás,Grêmio,3.0,2.0,"DOM, 08/12 16:00",Serra Dourada,38,2019
559,CSA,São Paulo,1.0,2.0,"DOM, 08/12 16:00",Rei Pelé,38,2019


In [29]:
df = pd.read_csv('jogadores.csv')

# df = df[df['ano'].isin([2020])]
df = df[df['rodada']>7]

df['id']  = df['Nome'].apply( lambda x: hashlib.sha1(str.encode(x)).hexdigest() )
df['pos'] = df['Nome'].apply( lambda x: x.split('(')[-1].split(')')[0].strip() )

df[['Nome', 'id', 'pos']]

,Nome,id,pos
1544,Marinho (ATA),79471fa35f3143ed842e45095f0f50314476b2a9,ATA
1545,Thiago Galhardo (MEI),d7cf1f1ebda261e1c21956a488ece0e62e519435,MEI
1546,Saravia (LAT),09a1951987d2f8dd0c99f3faad7d03d07af8873d,LAT
1547,João Paulo (GOL),eecc56b868165d01bf2a7db97c41e2ed358e2e6b,GOL
1548,Arrascaeta (MEI),a0fce35353de77b52ad41ac388bf917a1cae2b77,MEI
...,...,...,...
12291,Lucas Hernández (LAT),14f33434489bd23622b395e431e4661597182291,LAT
12292,Luccas Claro (ZAG),dc087e601838e0fcb99b470960ae8ec1a254c9e4,ZAG
12293,Helinho (ATA),5312766b3b17ef133c535874609315240162e5e0,ATA
12294,Wesley (MEI),a54d28bd16bab84b30ecead11c1ce39402aa49c6,MEI


In [30]:
df.iloc[-51]

Nome                                         Léo (ZAG)
Clube                                         Cruzeiro
Preço                                             6.09
J                                                   18
Média                                             1.78
Últ. Pont.                                         1.5
Variação                                         -0.42
DS                                                 NaN
G                                                  NaN
A                                                  NaN
SG                                                   3
FS                                                   6
FF                                                   1
FD                                                   1
FT                                                 NaN
DD                                                 NaN
DP                                                 NaN
GC                                                 NaN
CV        

# modelos

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import normalize, StandardScaler
from sklearn.metrics         import classification_report
from boruta                  import BorutaPy

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble     import RandomForestClassifier

## regressão logística binária

In [138]:
train_x = df[['Preço', 'Média','Variação','DS','G','A','SG','FS','FF','FD','FT','DD','DP','GC','CV','CA','PP','GS','FC','I','PI','RB','PE']].fillna(0.0)
train_y = np.where(df['pos']=='ATA', True, False)

_cols = train_x.columns.to_list()
# train_x = pd.DataFrame(StandardScaler().fit_transform(train_x[_cols]), columns=_cols)
train_x = pd.DataFrame(normalize(train_x[_cols]), columns=_cols)

In [139]:
X_train, X_holdout, y_train, y_holdout = train_test_split(train_x, train_y, test_size = 0.2, random_state = 37)

In [140]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=1, random_state=1, max_iter=23)

feat_selector.fit(X_train.values, y_train)

Iteration: 1 / 23
Iteration: 2 / 23
Iteration: 3 / 23
Iteration: 4 / 23
Iteration: 5 / 23
Iteration: 6 / 23
Iteration: 7 / 23
Iteration: 8 / 23
Iteration: 9 / 23
Iteration: 10 / 23
Iteration: 11 / 23
Iteration: 12 / 23
Iteration: 13 / 23
Iteration: 14 / 23
Iteration: 15 / 23
Iteration: 16 / 23
Iteration: 17 / 23
Iteration: 18 / 23
Iteration: 19 / 23
Iteration: 20 / 23
Iteration: 21 / 23
Iteration: 22 / 23


BorutaPy finished running.

Iteration: 	23 / 23
Confirmed: 	21
Tentative: 	0
Rejected: 	1


BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                          class_weight='balanced',
                                          criterion='gini', max_depth=5,
                                          max_features='auto',
                                          max_leaf_nodes=None, max_samples=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=132, n_jobs=-1,
                                          oob_score=False,
                                          random_state=RandomState(MT19937) at 0x7F5B56986780,
                                          verbose=0, warm_start=

In [141]:
pd.concat( [X_train.columns.to_frame().reset_index()['index'], pd.DataFrame(feat_selector.ranking_)], axis=1).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
index,Preço,Média,Variação,DS,G,A,SG,FS,FF,FD,FT,DD,DP,GC,CV,CA,PP,GS,FC,I,PI,RB,PE
0,1,1,1,1,1,1,1,1,1,1,1,1,2,1,2,1,1,1,1,1,1,1,1


In [142]:
_cols = X_train.columns[ feat_selector.support_]
clf = LogisticRegression(random_state=0).fit(X_train[_cols], y_train)

y_pred_holdout = clf.predict(X_holdout[_cols])
print( classification_report(y_holdout, y_pred_holdout, target_names=['False', 'True']) )

              precision    recall  f1-score   support

       False       0.92      0.97      0.95      1501
        True       0.84      0.64      0.73       343

    accuracy                           0.91      1844
   macro avg       0.88      0.81      0.84      1844
weighted avg       0.91      0.91      0.91      1844



In [143]:
pd.concat( [pd.DataFrame(_cols), 
            pd.DataFrame(clf.coef_).T], axis=1)

,0,0
0,Preço,1.953021
1,Média,0.135339
2,Variação,0.309432
3,DS,-3.730350
4,G,5.965869
5,A,2.154152
6,SG,-7.123393
7,FS,4.585971
8,FF,5.230703
9,FD,4.042832


## regressão logística multi-classe

In [144]:
train_x = df[['Preço', 'Média','Variação','DS','G','A','SG','FS','FF','FD','FT','DD','DP','GC','CV','CA','PP','GS','FC','I','PI','RB','PE']].fillna(0.0)
train_y = df['pos']#np.where(df['pos']=='ATA', True, False)

_cols = train_x.columns.to_list()
# train_x = pd.DataFrame(StandardScaler().fit_transform(train_x[_cols]), columns=_cols)
train_x = pd.DataFrame(normalize(train_x[_cols]), columns=_cols)

In [146]:
X_train, X_holdout, y_train, y_holdout = train_test_split(train_x, train_y, stratify=train_y, test_size = 0.2, random_state = 37)

In [150]:
display(pd.DataFrame(y_train)['pos'].value_counts(normalize=True))
display(pd.DataFrame(y_holdout)['pos'].value_counts(normalize=True))

MEI    0.312000
ATA    0.188339
ZAG    0.169220
LAT    0.158102
TEC    0.086508
GOL    0.085831
Name: pos, dtype: float64

MEI    0.311822
ATA    0.188720
ZAG    0.169197
LAT    0.158351
TEC    0.086226
GOL    0.085683
Name: pos, dtype: float64

In [151]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=1, random_state=1, max_iter=23)

feat_selector.fit(X_train.values, y_train)

Iteration: 1 / 23
Iteration: 2 / 23
Iteration: 3 / 23
Iteration: 4 / 23
Iteration: 5 / 23
Iteration: 6 / 23
Iteration: 7 / 23
Iteration: 8 / 23
Iteration: 9 / 23
Iteration: 10 / 23
Iteration: 11 / 23
Iteration: 12 / 23
Iteration: 13 / 23
Iteration: 14 / 23
Iteration: 15 / 23
Iteration: 16 / 23
Iteration: 17 / 23
Iteration: 18 / 23
Iteration: 19 / 23


BorutaPy finished running.

Iteration: 	20 / 23
Confirmed: 	23
Tentative: 	0
Rejected: 	0


BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                          class_weight='balanced',
                                          criterion='gini', max_depth=5,
                                          max_features='auto',
                                          max_leaf_nodes=None, max_samples=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=135, n_jobs=-1,
                                          oob_score=False,
                                          random_state=RandomState(MT19937) at 0x7F5B56986A98,
                                          verbose=0, warm_start=

In [152]:
pd.concat( [X_train.columns.to_frame().reset_index()['index'], pd.DataFrame(feat_selector.ranking_)], axis=1).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
index,Preço,Média,Variação,DS,G,A,SG,FS,FF,FD,FT,DD,DP,GC,CV,CA,PP,GS,FC,I,PI,RB,PE
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [160]:
_cols = X_train.columns[ feat_selector.support_]
clf = LogisticRegression(random_state=0).fit(X_train[_cols], y_train)

y_pred_holdout = clf.predict(X_holdout[_cols])
print( classification_report(y_holdout, y_pred_holdout, target_names=sorted(y_holdout.unique())) )

              precision    recall  f1-score   support

         ATA       0.75      0.68      0.72       348
         GOL       1.00      0.96      0.98       158
         LAT       0.71      0.54      0.62       292
         MEI       0.72      0.86      0.79       575
         TEC       0.89      1.00      0.94       159
         ZAG       0.73      0.68      0.70       312

    accuracy                           0.77      1844
   macro avg       0.80      0.79      0.79      1844
weighted avg       0.77      0.77      0.76      1844



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [182]:
_coefs = pd.concat( [pd.DataFrame(_cols), 
            pd.DataFrame(clf.coef_).T], axis=1)
_coefs.columns = _coefs.columns[:-6].to_list() + sorted(y_holdout.unique())
_coefs

,0,ATA,GOL,LAT,MEI,TEC,ZAG
0,Preço,-0.278003,-1.176171,-1.543337,-2.155060,5.023617,0.128955
1,Média,-0.734285,-1.238613,0.989994,-1.749860,3.739687,-1.006923
2,Variação,0.335143,-0.134636,-0.127180,0.386366,0.025014,-0.484707
3,DS,-2.106726,-4.760869,0.447867,3.857282,-0.514344,3.076791
4,G,4.992768,-0.463016,-4.575783,-0.513464,-0.463444,1.022939
5,A,1.885278,-0.330126,3.485229,0.122697,-0.267562,-4.895516
6,SG,-6.887074,1.974240,9.296098,-18.432989,-1.362341,15.412066
7,FS,4.775896,-3.034522,0.714315,1.905535,-2.771111,-1.590113
8,FF,5.995269,-1.921652,-3.941561,2.023937,-1.492529,-0.663465
9,FD,4.843438,-1.185683,-1.921749,2.775535,-1.357979,-3.153563


,ATA,GOL,LAT,MEI,TEC,ZAG
0,-0.278003,-0.734285,0.335143,-2.106726,4.992768,1.885278
1,-6.887074,4.775896,5.995269,4.843438,1.786632,-0.344002
2,-0.010121,-0.226094,0.220729,-3.027524,0.456040,-0.381206
3,3.333491,6.778112,0.396254,-2.697173,-0.617862,-1.176171
4,-1.238613,-0.134636,-4.760869,-0.463016,-0.330126,1.974240
5,-3.034522,-1.921652,-1.185683,-0.120252,6.665961,0.115130
6,-0.024864,0.007530,-0.145159,-0.020252,5.900913,-4.534283
7,-0.455364,0.830090,-1.264584,-0.640485,-1.543337,0.989994
8,-0.127180,0.447867,-4.575783,3.485229,9.296098,0.714315
9,-3.941561,-1.921749,-0.777058,-2.030580,-0.053907,-0.529213


In [162]:
gc.collect()

743